In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d ananthu017/emotion-detection-fer

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/emotion-detection-fer.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
import os

In [ ]:
train_d = '/content/train'
filepaths = []
labels = []

folds = os.listdir(train_d)
for fold in folds:
    foldpath = os.path.join(train_d, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)


Fseries = pd.Series(filepaths, name= 'paths')
Lseries = pd.Series(labels, name='labels')
train_ds = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
train_ds

,paths,labels
0,/content/train/fearful/im2661.png,fearful
1,/content/train/fearful/im1874.png,fearful
2,/content/train/fearful/im850.png,fearful
3,/content/train/fearful/im450.png,fearful
4,/content/train/fearful/im995.png,fearful
...,...,...
28704,/content/train/disgusted/im339.png,disgusted
28705,/content/train/disgusted/im421.png,disgusted
28706,/content/train/disgusted/im242.png,disgusted
28707,/content/train/disgusted/im205.png,disgusted


In [ ]:
test_d = '/content/test'
filepaths = []
labels = []

folds = os.listdir(test_d)
for fold in folds:
    foldpath = os.path.join(test_d, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)


Fseries = pd.Series(filepaths, name= 'paths')
Lseries = pd.Series(labels, name='labels')
test_ds = pd.concat([Fseries, Lseries], axis= 1)





In [ ]:
test_ds

,paths,labels
0,/content/test/fearful/im850.png,fearful
1,/content/test/fearful/im450.png,fearful
2,/content/test/fearful/im995.png,fearful
3,/content/test/fearful/im207.png,fearful
4,/content/test/fearful/im320.png,fearful
...,...,...
7173,/content/test/disgusted/im31.png,disgusted
7174,/content/test/disgusted/im13.png,disgusted
7175,/content/test/disgusted/im70.png,disgusted
7176,/content/test/disgusted/im107.png,disgusted


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
valid_ds, test_ds = train_test_split(test_ds,  train_size= 0.6, random_state= 66)

In [ ]:
train_ds = pd.DataFrame(train_ds)
valid_ds = pd.DataFrame(valid_ds)
test_ds = pd.DataFrame(test_ds)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_g = ImageDataGenerator()
ts_g = ImageDataGenerator()
train_g = tr_g.flow_from_dataframe( train_ds, x_col= 'paths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_g = ts_g.flow_from_dataframe( valid_ds, x_col= 'paths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_g = ts_g.flow_from_dataframe( test_ds, x_col= 'paths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)

Found 28709 validated image filenames belonging to 7 classes.
Found 4306 validated image filenames belonging to 7 classes.
Found 2872 validated image filenames belonging to 7 classes.


In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adamax
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_g.class_indices.keys()))

base_model = tf.keras.applications.efficientnet.EfficientNetB7(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')

model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional  (None, 2560)              64097687  
 )                                                               
                                                                 
 batch_normalization_2 (Bat  (None, 2560)              10240     
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 256)               655616    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 64765342 (247.06 MB)
Trainable params: 6

In [ ]:
batch_size = 18
epochs =  8

history = model.fit(x= train_g, epochs= epochs, verbose= 1, validation_data= valid_g,
                    validation_steps= None, shuffle= False)

In [ ]:
train_score = model.evaluate(train_g, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_g, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_g, steps= test_steps, verbose= 1)